# News API

A Python client for the [News API](https://newsapi.org/docs/).


## License

Provided under [MIT License](https://github.com/mattlisiv/newsapi-python/blob/master/LICENSE.txt) by Matt Lisivick.

```
THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS
FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR
COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER
IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN
CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.
```


## General

This is a Python client library for [News API V2](https://newsapi.org/).
The functions and methods for this library should mirror the
endpoints specified by the News API [documentation](https://newsapi.org/docs/endpoints).


## Installation

Installation for the package can be done via `pip`:

In [2]:
%pip install newsapi-python
%pip install opencv-python
%pip install mediapipe
%pip install screeninfo
%pip install cvzone
%pip install matplotlib
%pip install pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


...


[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Usage

After installation, import the client class into your project:

In [3]:
from newsapi import NewsApiClient

Initialize the client with your API key:

In [4]:
import os

api = NewsApiClient(api_key=os.environ['NEWSAPI_ORG'])


### Endpoints

An instance of `NewsApiClient` has three instance methods corresponding to three News API endpoints.

#### Top Headlines

Use `.get_top_headlines()` to pull from the [`/top-headlines`](https://newsapi.org/docs/endpoints/top-headlines) endpoint:

In [5]:
api.get_top_headlines(sources='bbc-news')

{'status': 'ok',
 'totalResults': 10,
 'articles': [{'source': {'id': 'bbc-news', 'name': 'BBC News'},
   'author': 'BBC News',
   'title': 'Beijing: Twelve held after Beijing hospital fire kills 29',
   'description': 'The director and deputy director of the hospital are among those being questioned by police.',
   'url': 'http://www.bbc.co.uk/news/world-asia-china-65320160',
   'urlToImage': 'https://ichef.bbci.co.uk/news/1024/branded_news/6766/production/_129407462_9966e80b03fba227f480471b191392c8331913140_0_4145_27631000x667.jpg',
   'publishedAt': '2023-04-19T07:07:15.2949366Z',
   'content': 'Twelve people have been detained by police for questioning after a hospital fire in Beijing killed at least 29, most of them patients. \r\nThe fire, among the deadliest in recent years, broke out in Ch… [+2225 chars]'},
  {'source': {'id': 'bbc-news', 'name': 'BBC News'},
   'author': 'BBC News',
   'title': 'Ukraine war: The Russian ships accused of North Sea sabotage',
   'description': 'D

#### Everything

Use `.get_everything()` to pull from the [`/everything`](https://newsapi.org/docs/endpoints/everything) endpoint:

In [6]:
api.get_everything(q='bitcoin')

{'status': 'ok',
 'totalResults': 12821,
 'articles': [{'source': {'id': None, 'name': 'Gizmodo.com'},
   'author': 'Kyle Barr',
   'title': 'Here’s How to Find the Original Bitcoin Manifesto in Your Mac',
   'description': 'An Apple developer left one of the strangest Easter eggs hidden in the macOS, offering a surprising and baffling connection between the Cupertino tech company’s mainline operating system to everybody’s (least) favorite cryptographic digital currency. Read mor…',
   'url': 'https://gizmodo.com/bitcoin-apple-mac-bitcoin-white-paper-1850307069',
   'urlToImage': 'https://i.kinja-img.com/gawker-media/image/upload/c_fill,f_auto,fl_progressive,g_center,h_675,pg_1,q_80,w_1200/176f6f215c9c45306ca8269f8232701e.jpg',
   'publishedAt': '2023-04-06T15:30:00Z',
   'content': 'An Apple developer left one of the strangest Easter eggs hidden in the macOS, offering a surprising and baffling connection between the Cupertino tech companys mainline operating system to everybodys… [+38

#### Sources

Use `.get_sources()` to pull from the [`/sources`](https://newsapi.org/docs/endpoints/sources) endpoint:

In [7]:
api.get_sources()

{'status': 'ok',
 'sources': [{'id': 'abc-news',
   'name': 'ABC News',
   'description': 'Your trusted source for breaking news, analysis, exclusive interviews, headlines, and videos at ABCNews.com.',
   'url': 'https://abcnews.go.com',
   'category': 'general',
   'language': 'en',
   'country': 'us'},
  {'id': 'abc-news-au',
   'name': 'ABC News (AU)',
   'description': "Australia's most trusted source of local, national and world news. Comprehensive, independent, in-depth analysis, the latest business, sport, weather and more.",
   'url': 'http://www.abc.net.au/news',
   'category': 'general',
   'language': 'en',
   'country': 'au'},
  {'id': 'aftenposten',
   'name': 'Aftenposten',
   'description': 'Norges ledende nettavis med alltid oppdaterte nyheter innenfor innenriks, utenriks, sport og kultur.',
   'url': 'https://www.aftenposten.no',
   'category': 'general',
   'language': 'no',
   'country': 'no'},
  {'id': 'al-jazeera-english',
   'name': 'Al Jazeera English',
   'descr

## For Windows users printing to _cmd_ or _powershell_

You will encounter an error if you attempt to print the .json() object to the command line. This is because the '{', '}' curly braces to be printed to the console.
This becomes especially annoying if developers wish to get 'under the hood'.

Here is the error:

```
UnicodeEncodeError: 'charmap' codec can't encode character '\u2019' in position 1444: character maps to <undefined>
```

This can be fixed by:
    - installing 'win-unicode-console'
        `py -mpip install win-unicode-console`
    - then running it while calling your python script...
        `py -mrun myPythonScript.py`

Another option is hardcoding your console to only print in utf-8. This is a bad idea, as it could ruin many other scripts and/or make errors MUCH more difficult to track.
[More information](https://stackoverflow.com/questions/5419/python-unicode-and-the-windows-console/32176732#32176732).


## Showtime - Let's get some news but using it together with OpenCV and Pandas


In [8]:
import cv2
import pandas as pd
import numpy as np

In [9]:
# init
api = NewsApiClient(api_key=os.environ['NEWSAPI_ORG'])

In [10]:
def get_top_headlines():
    response = api.get_top_headlines(
        sources='bbc-news, cnn, fox-news, google-news, the-new-york-times, the-wall-street-journal, the-washington-post, time, usa-today, wired'
    )
    return response


top_headlines = get_top_headlines()

In [11]:
print(
    """
    Top Headlines
    -------------
    {}
    """.format(top_headlines)
)


    Top Headlines
    -------------
    {'status': 'ok', 'totalResults': 90, 'articles': [{'source': {'id': 'fox-news', 'name': 'Fox News'}, 'author': 'Ashley Hume, Larry Fink', 'title': "Pat Boone's concern for America: ‘We’re going down the tubes morally'", 'description': 'Pat Boone, who is celebrating his 70th year in show business, reflected on moral decay in America. He slammed Hollywood for glorifying immoral behavior.', 'url': 'https://www.foxnews.com/entertainment/pat-boones-concern-america-going-down-the-tubes-morally', 'urlToImage': 'https://static.foxnews.com/foxnews.com/content/uploads/2023/04/pat.jpg', 'publishedAt': '2023-04-19T08:00:23Z', 'content': 'Pat Boone has some serious concerns about the state of morality in America.\r\nThe singer and actor, who is celebrating his 70th year in show business, was one of the most famous teen idols of the 1950… [+5808 chars]'}, {'source': {'id': 'the-wall-street-journal', 'name': 'The Wall Street Journal'}, 'author': 'Paul Hannon',

In [12]:
def news_to_df(news):
    # map all the articles to a list
    articles = list(map(lambda x: x['title'], news['articles']))

    # create a dataframe
    df = pd.DataFrame(articles, columns=['title'])

    # add a column for the length of the title
    df['title_length'] = df['title'].apply(lambda x: len(x))

    # add a column for the number of words in the title
    df['title_words'] = df['title'].apply(lambda x: len(x.split(' ')))

    return df

In [13]:
df = news_to_df(top_headlines)

In [14]:
display(df)

,title,title_length,title_words
0,Pat Boone's concern for America: ‘We’re going ...,69,11
1,"U.K. Inflation Stays Above 10%, Raising Prospe...",83,11
2,Beijing: Twelve held after Beijing hospital fi...,57,9
3,Two-time 'Survivor' contestant Keith Nale dies...,52,8
4,Florida swim instructor's viral TikTok warns p...,108,17
5,AI chatbot 'hallucinations' perpetuate politic...,104,12
6,"Rep. Lee endorses DeSantis, bringing Capitol H...",110,18
7,Ukraine war: The Russian ships accused of Nort...,60,10
8,"A quarter of Americans live with polluted air,...",142,23
9,Kaylin Gillis: Driveway shooting suspect shows...,66,10


In [15]:
import cv2
import mediapipe as mp
import cvzone
import numpy as np
import screeninfo


In [16]:
# Using OpenCV to display the image
cap = cv2.VideoCapture(0)

cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
cap.set(cv2.CAP_PROP_FPS, 60)

# text as one line string
text = ' '.join(df['title'].tolist())

# add '###' between each title
text = ' ... | '.join(df['title'].tolist())

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

zoomed_text_color = (0, 255, 0)
standard_text_color = (255, 255, 255)
color = standard_text_color

direction = 0
font_size = 12
box_size = 50

with mp_hands.Hands(
        model_complexity=0,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as hands:

    while cap.isOpened():

        success, image = cap.read()

        if not success:
            print("Ignoring empty camera frame.")
            # If loading a video, use 'break' instead of 'continue'.
            continue

        image = cv2.flip(image, 1)

        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = hands.process(image)

        # # Draw the hand annotations on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # create a background for the text with a border
        cv2.rectangle(image, (0, image.shape[0] - box_size), (image.shape[1], image.shape[0]), (0, 0, 0), -1)
        # border
        cv2.rectangle(image, (0, image.shape[0] - box_size), (image.shape[1], image.shape[0]), (255, 255, 255), 2)


        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                # mp_drawing.draw_landmarks(
                #     image,
                #     hand_landmarks,
                #     mp_hands.HAND_CONNECTIONS,
                #     mp_drawing_styles.get_default_hand_landmarks_style(),
                #     mp_drawing_styles.get_default_hand_connections_style())

                # if left hand is raised then move the text to the left
                if hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].x < 0.2 and len(results.multi_hand_landmarks) == 1:
                    text = text[1:] + text[0]
                    direction = 0
                    font_size = 12
                    color = standard_text_color
                    box_size = 50

                # if right hand is raised then move the text to the right
                elif hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].x > 0.8 and len(results.multi_hand_landmarks) == 1:
                    direction = 1
                    font_size = 12
                    color = standard_text_color
                    box_size = 50

                # if both hands are raised then increase the font size to 36pt and change the color
                elif 0.2 < hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].x < 0.8 and len(results.multi_hand_landmarks) == 2:
                    font_size = 36
                    color = zoomed_text_color
                    box_size = 100

        else:
            # if no hands are detected then move the text to the left
            font_size = 12
            color = standard_text_color
            box_size = 50

        if direction == 0:
            text = text[1:] + text[0]
        elif direction == 1:
            text = text[-1] + text[:-1]

        # draw the text
        cv2.putText(image, text, (2, image.shape[0] - 10), cv2.FONT_HERSHEY_SIMPLEX, font_size / 12, color, 2)

        # put a logo in the top left corner
        logo = cv2.imread('logo.png')

        # resize the logo
        logo = cv2.resize(logo, (int(logo.shape[1] / 12), int(logo.shape[0] / 12)))

        # add the logo to the image
        image[0:logo.shape[0], 0:logo.shape[1]] = logo

        # Flip the image horizontally for a selfie-view display.
        cv2.imshow('AI News', image)

        # wait for the 'q' key to be pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        # if 's' is pressed, save the image
        if cv2.waitKey(1) & 0xFF == ord('s'):
            cv2.imwrite('news.jpg', image)

cap.release()

# stop the video
cv2.destroyAllWindows()
